In [ ]:
import os
from pathlib import Path

In [ ]:
# for colab
# !git clone https://github.com/laralex/Sk-DL2021-FinalProject
# repo_dir = Path().absolute()/'Sk-DL2021-FinalProject'
# %pushd Sk-DL2021-FinalProject
# !git pull
# !git checkout permanent_generation
# !pip install pytorch_lightning
# import sys
# sys.path.append('Sk-DL2021-FinalProject')

In [ ]:
# for local
import sys
sys.path.append('..')
repo_dir = Path().absolute().parent

In [ ]:
!pwd

import torch
from data.split_step_generator import SplitStepGenerator, find_dataset_subdir

GOOGLE_DRIVE = False

if GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount(f'/content/drive')
    root_dir = Path('/content/drive/drive/MyDrive/Sk-DL2021-Datasets')
else:
    root_dir = repo_dir.parent / 'generated_datasets'
if not os.path.exists(root_dir):
    os.makedirs(root_dir, exist_ok=True)
    
root_dir

In [ ]:
import yaml
CONFIG = f'{repo_dir}/configs/DebugDataset2D.yaml'
BATCH_SIZE = 1
GENERATE_TRAIN_BATCHES = 2
GENERATE_VAL_BATCHES = 0
GENERATE_TEST_BATCHES = 0

if os.path.exists(CONFIG):
    with open(CONFIG, 'r') as stream:
        config_hparams = yaml.safe_load(stream)['data']['init_args']
        config_hparams['data_source_type'] = 'generation'
        config_hparams['load_dataset_root_path'] = None
        config_hparams['batch_size'] = BATCH_SIZE
        config_hparams['generate_n_train_batches'] = GENERATE_TRAIN_BATCHES
        config_hparams['generate_n_val_batches'] = GENERATE_VAL_BATCHES
        config_hparams['generate_n_test_batches'] = GENERATE_TEST_BATCHES
else:
    print('Config file cant be found')

In [ ]:
data_gen = SplitStepGenerator(**config_hparams)
config_hparams

In [ ]:
import yaml
import datetime 

NEW_DIR_NAME = 'high_nonlinearity'

def create_destination(hparams, datasets_root, new_dir_name=None):
    if new_dir_name is None:
        new_dir = root_dir/datetime.datetime.now().strftime("%m-%d-%Y=%H-%M-%S")
    else:
        new_dir = root_dir/new_dir_name
    os.makedirs(new_dir)
    assert not os.path.exists(f'{new_dir}/signal_hparams.yaml')
    with open(f'{new_dir}/signal_hparams.yaml', 'w') as outfile:
        yaml.dump(hparams, outfile, default_flow_style=False)
    return new_dir
    
destination_root = find_dataset_subdir(data_gen.signal_hparams, root_dir)
if destination_root is None:
    destination_root = create_destination(data_gen.signal_hparams, root_dir, NEW_DIR_NAME)
print('Destination: ', destination_root)       

In [ ]:
# make folders structure
def save_tensor(tensor, subdir):
    if tensor is None:
        print('Nothing to save', subdir)
        return
    if tensor.numel() == 0:
        return
    i = 0
    while os.path.exists(subdir/f"{i}.pt"):
        i += 1
    destination_path = subdir/f"{i}.pt"
    torch.save(torch.tensor([]), destination_path)
    torch.save(tensor.clone(), destination_path)
    
type_subdirs = [destination_root/sub for sub in ['train', 'val', 'test']]
for d in type_subdirs:
    os.makedirs(d, exist_ok=True)

In [ ]:
# generate and save
N_REPEATS = 3

for _ in range(N_REPEATS):
    data_gen.prepare_data()
    data_gen.setup()

    save_tensor(data_gen.train, type_subdirs[0])
    save_tensor(data_gen.val, type_subdirs[1])
    save_tensor(data_gen.test, type_subdirs[2])

In [ ]:
config_hparams['data_source_type'] = 'filesystem'
config_hparams['load_dataset_root_path'] = root_dir
config_hparams['batch_size'] = 2
data_gen_load = SplitStepGenerator(**config_hparams)
config_hparams

In [ ]:
data_gen_load.prepare_data()

In [ ]:
if data_gen_load.train is not None:
    print(data_gen_load.train.shape)